In [2]:
!ls /data/raw

NINJAV_S001_S001_T003.MOV


In [132]:
import re
import os
from os import path
from pathlib import Path
import hachoir
from datetime import timedelta, datetime, time
from pydantic import BaseModel, Field

In [133]:
vidfile = "/data/raw/NINJAV_S001_S001_T003.MOV"
assert path.exists(vidfile)

In [118]:
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata


class RawVideoMetaData(BaseModel):
    path: Path
    duration: timedelta
    created_on: datetime
    width: int
    height: int

def parse_metadata(fname) -> RawVideoMetaData:
    parser = createParser(fname)
    with parser:
        metadata = extractMetadata(parser).exportDictionary()['Metadata']
    metadata
    
    aa = re.search("(\d+) min (\d+) sec (\d+) ms", metadata['Duration'])
    minutes, secs, msecs = aa.groups()
    duration = timedelta(minutes=int(minutes), seconds=int(secs), milliseconds=int(msecs))

    created_on = datetime.fromisoformat('2024-06-18 06:13:24')

    width = re.match("(\d+) pixels", metadata['Image width']).groups()[0]
    width = int(width)

    height = re.match("(\d+) pixels", metadata['Image height']).groups()[0]
    height = int(height)
    
    return RawVideoMetaData(
        path=Path(fname),
        duration = duration,
        created_on = created_on,
        width=width,
        height=height
    )

metadata = parse_metadata(fname=vidfile)
metadata

RawVideoMetaData(path=PosixPath('/data/raw/NINJAV_S001_S001_T003.MOV'), duration=datetime.timedelta(seconds=3225, microseconds=760000), created_on=datetime.datetime(2024, 6, 18, 6, 13, 24), width=1920, height=20)

In [125]:

assert RawVideoMetaData.parse_raw(metadata.json()) == metadata
metadata.json()

'{"path": "/data/raw/NINJAV_S001_S001_T003.MOV", "duration": 3225.76, "created_on": "2024-06-18T06:13:24", "width": 1920, "height": 20}'

In [131]:
save_path = Path(".") / Path(vidfile).with_suffix('.json').name
save_path.write_text(metadata.json())

134